In [1]:

import pandas as pd

df = pd.read_csv(r"C:\\Users\\JellevanAs\\Documents\\Studie\\Thesis\\df-W2V_cat.csv")

df.columns.tolist()



['Unnamed: 0',
 'numerical_price',
 'word2vec_embeddings',
 'num_bedrooms',
 'num_rooms',
 'building_type_Bestaande bouw',
 'building_type_Nieuwbouw',
 'building_type_na',
 'tag_k.k.',
 'tag_v.o.n.',
 'house_category_Appartement',
 'house_category_Bungalow',
 'house_category_Eengezinswoning',
 'house_category_Grachtenpand',
 'house_category_Herenhuis',
 'house_category_Landhuis',
 'house_category_Other',
 'house_category_Unknown',
 'house_category_Villa',
 'house_category_Woonboerderij',
 'house_category_Woonboot',
 'energy_label_encoded',
 'size_scaled',
 'longitude_scaled',
 'latitude_scaled']

In [2]:
df = df.drop(df.columns[0], axis=1)
df.columns.tolist()

['numerical_price',
 'word2vec_embeddings',
 'num_bedrooms',
 'num_rooms',
 'building_type_Bestaande bouw',
 'building_type_Nieuwbouw',
 'building_type_na',
 'tag_k.k.',
 'tag_v.o.n.',
 'house_category_Appartement',
 'house_category_Bungalow',
 'house_category_Eengezinswoning',
 'house_category_Grachtenpand',
 'house_category_Herenhuis',
 'house_category_Landhuis',
 'house_category_Other',
 'house_category_Unknown',
 'house_category_Villa',
 'house_category_Woonboerderij',
 'house_category_Woonboot',
 'energy_label_encoded',
 'size_scaled',
 'longitude_scaled',
 'latitude_scaled']

In [3]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split


df["word2vec_embeddings"] = df["word2vec_embeddings"].apply(literal_eval).apply(np.array)

# Separating the target variable and embeddings
x = df["word2vec_embeddings"]
y = df['numerical_price']

# Dropping the 'ada_embedding_eng' column from df
z = df.drop(["word2vec_embeddings", "numerical_price"], axis=1)


# Deleting original dataframe for memory purpose
del df

# Explode the embedding arrays into separate columns
x = x.apply(pd.Series)

# Concatenating the exploded embeddings with the rest of the data
concatenated_df = pd.concat([x, z], axis=1).reset_index(drop=True)
concatenated_df.columns = concatenated_df.columns.astype(str)

x_train, x_test, y_train, y_test = train_test_split(concatenated_df, y, test_size=0.2, random_state=42)

In [4]:
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Ensure y_train and y_test are 1D arrays
# This step is only necessary if y_train or y_test are not already 1D arrays
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

# Check the shapes again after any necessary adjustments
print("Adjusted y_train shape:", y_train.shape)
print("Adjusted y_test shape:", y_test.shape)

Adjusted y_train shape: (57610,)
Adjusted y_test shape: (14403,)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, explained_variance_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import numpy as np

def train_and_evaluate_nn(X_train, y_train):
    # Basic hyperparameters
    learning_rate = 0.001
    neurons_layer_1 = 1024
    neurons_layer_2 = 512
    neurons_layer_3 = 256
    batch_size = 32
    epochs = 10

    # KFold Cross-validation
    kf = KFold(n_splits=5)
    r2_scores, mse_scores, rmse_scores, mape_scores, ev_scores = [], [], [], [], []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        # Build model
        model = Sequential()
        model.add(Dense(neurons_layer_1, activation='relu', input_shape=(X_train_fold.shape[1],)))
        model.add(Dense(neurons_layer_2, activation='relu'))
        model.add(Dense(neurons_layer_3, activation='relu'))
        model.add(Dense(1, activation='linear'))

        # Compile model
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='mean_squared_error')

        # Scaling y_train
        scaler = StandardScaler()
        y_train_scaled = scaler.fit_transform(y_train_fold.reshape(-1, 1)).flatten()

        # Train model
        model.fit(X_train_fold, y_train_scaled, epochs=epochs, batch_size=batch_size, verbose=0)

        # Predict and evaluate
        y_pred_scaled = model.predict(X_test_fold)
        y_pred = scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

        # Metrics calculation
        r2 = r2_score(y_test_fold, y_pred)
        mse = mean_squared_error(y_test_fold, y_pred)
        rmse = np.sqrt(mse)
        mape = mean_absolute_percentage_error(y_test_fold, y_pred)
        ev = explained_variance_score(y_test_fold, y_pred)

        # Append scores
        r2_scores.append(r2)
        mse_scores.append(mse)
        rmse_scores.append(rmse)
        mape_scores.append(mape)
        ev_scores.append(ev)

    # Calculate average scores
    avg_r2 = np.mean(r2_scores)
    avg_mse = np.mean(mse_scores)
    avg_rmse = np.mean(rmse_scores)
    avg_mape = np.mean(mape_scores)
    avg_ev = np.mean(ev_scores)

    return avg_r2, avg_mse, avg_rmse, avg_mape, avg_ev


In [6]:
# Assuming x_train and y_train are already defined and preprocessed

# Call the function with your data
average_r2_score, average_mse, average_rmse, average_mape, average_ev = train_and_evaluate_nn(x_train, y_train)

# Print the performance metrics
print("Average R2 Score:", average_r2_score)
print("Average MSE:", average_mse)
print("Average RMSE:", average_rmse)
print("Average MAPE:", average_mape)
print("Average Explained Variance:", average_ev)


361/361 [==============================] - 1s 4ms/step
Average R2 Score: 0.7552615431509915
Average MSE: 42387845000.0
Average RMSE: 205291.64
Average MAPE: 0.19541392
Average Explained Variance: 0.7578089594841003
